Install conda 

In [ ]:
%%bash
wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
bash Miniconda3-py39_4.12.0-Linux-x86_64.sh
apt update
apt install parallel bcftools

Download GFF3

In [2]:
%%bash
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_19/gencode.v19.annotation.gff3.gz
gzip -d gencode.v19.annotation.gff3.gz

--2022-10-11 04:42:43--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_19/gencode.v19.annotation.gff3.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46459624 (44M) [application/x-gzip]
Saving to: ‘gencode.v19.annotation.gff3.gz’

     0K .......... .......... .......... .......... ..........  0%  591K 77s
    50K .......... .......... .......... .......... ..........  0%  590K 77s
   100K .......... .......... .......... .......... ..........  0%  592K 77s
   150K .......... .......... .......... .......... ..........  0%  597K 76s
   200K .......... .......... .......... .......... ..........  0%  595K 76s
   250K .......... .......... .......... .......... ..........  0% 30.4M 64s
   300K .......... .......... .......... .......... ..........  0%  596K 65s
   350K .......... .......... .......... .......... ..........  

#genes
ALS2 ANG CHMP2B DAO DCTN1 FIG4 FUS NEFH OPTN PFN1 PON1 PON2   PON3 PRPH SETX  SOD1  SQSTM1 TARDBP TREM2 UBQLN2  VAPB VCPVEGFA 

the below code is to generate the regions of all those genes

In [4]:
%%bash
parallel --gnu " ./GeneTarget/getRegions.sh {} > {}.regions  " ::: ALS2 ANG CHMP2B DAO DCTN1 FIG4 FUS NEFH OPTN PFN1 PON1 PON2   PON3 PRPH SETX  SOD1  SQSTM1 TARDBP TREM2 UBQLN2  VAPB VCPVEGFA

In [5]:
%%bash
wc -l *regions

  292 ALS2.regions
   22 ANG.regions
   60 CHMP2B.regions
  142 DAO.regions
  651 DCTN1.regions
  142 FIG4.regions
  190 FUS.regions
   14 NEFH.regions
  255 OPTN.regions
   30 PFN1.regions
   78 PON1.regions
  173 PON2.regions
  152 PON3.regions
   73 PRPH.regions
  222 SETX.regions
   43 SOD1.regions
  194 SQSTM1.regions
  137 TARDBP.regions
   44 TREM2.regions
    8 UBQLN2.regions
   59 VAPB.regions
    0 VCPVEGFA.regions
 2981 total


Download VCF files from the project

In [ ]:
%%bash
dx download SV/*

In [7]:
%%bash

echo "ALS2 ANG CHMP2B DAO DCTN1 FIG4 FUS NEFH OPTN PFN1 PON1 PON2   PON3 PRPH SETX  SOD1  SQSTM1 TARDBP TREM2 UBQLN2  VAPB VCPVEGFA"| tr -s ' ' $'\n' > genes
ls GeneTarget/data/ | grep -P "gz$" > samples
parallel --gnu 'bcftools view -R {1}.regions GeneTarget/data/{2} > events/{1}_{2} ' :::: genes :::: samples
                                

Generate the table.csv

In [20]:
from os import listdir
from os.path import isfile, join
mypath='events/'

data={}
genes=set()
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for fileName in onlyfiles:
    i=fileName.find("_")
    gene=fileName[:i]
    sample=fileName[i+1:]
    genes.add(gene)
    lines= [l.strip() for l in open('events/'+fileName)]
    svs=filter(lambda l: len(l)>0 and l[0]!='#',lines)
    numEvents=len(list(svs))
    if sample not in data:
        data[sample]={}
    data[sample][gene]=numEvents

genes=list(genes)
outputFile=open("out.csv",'w')
header=[sample] + genes
outputFile.write(",".join(header)+"\n")

for sample, events in data.items():
    line=[sample]+[str(events[g]) for g in genes]
    outputFile.write(",".join(line)+"\n")
    